In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [5]:
df = pd.read_csv("../data.csv").dropna()

In [6]:
df

,date,time_of_day,weight
0,2022-12-08,morning,93.6
1,2022-12-08,evening,94.3
2,2022-12-09,morning,92.4
3,2022-12-10,evening,94.4
4,2022-12-11,morning,93.6
5,2022-12-11,evening,95.6
6,2022-12-12,morning,93.9
7,2022-12-12,evening,94.8
8,2022-12-13,morning,93.5
9,2022-12-13,evening,94.6


In [7]:
def moving_average(arr, window_size):
    return np.convolve(arr, np.ones(window_size), "valid") / window_size

In [120]:
moving_average(df["weight"], 7)

array([93.97142857, 94.14285714, 94.02857143, 94.34285714, 94.21428571])

In [ ]:
diet_start = datetime.date(2022, 12, 13)

In [46]:
df["date"] = pd.to_datetime(df["date"]).dt.date

In [51]:
df[df["date"] == diet_start]["weight"]

93.5

In [52]:
start_avg_weight = (
    df[df["date"] == diet_start]["weight"].iloc[0]
    + df[df["date"] == diet_start]["weight"].iloc[1]
) / 2

In [53]:
start_avg_weight

94.05

In [54]:
x_start = diet_start
x_end = diet_start + datetime.timedelta(days=100)
x = pd.date_range(start=x_start, end=x_end)
weekly_coefficient = 0.5
daily_coefficient = 0.5 / 7
end_avg_weight = start_avg_weight - (daily_coefficient * len(x))

In [58]:
y = np.linspace(start_avg_weight, end_avg_weight, len(x))

In [59]:
y

array([94.05      , 93.97785714, 93.90571429, 93.83357143, 93.76142857,
       93.68928571, 93.61714286, 93.545     , 93.47285714, 93.40071429,
       93.32857143, 93.25642857, 93.18428571, 93.11214286, 93.04      ,
       92.96785714, 92.89571429, 92.82357143, 92.75142857, 92.67928571,
       92.60714286, 92.535     , 92.46285714, 92.39071429, 92.31857143,
       92.24642857, 92.17428571, 92.10214286, 92.03      , 91.95785714,
       91.88571429, 91.81357143, 91.74142857, 91.66928571, 91.59714286,
       91.525     , 91.45285714, 91.38071429, 91.30857143, 91.23642857,
       91.16428571, 91.09214286, 91.02      , 90.94785714, 90.87571429,
       90.80357143, 90.73142857, 90.65928571, 90.58714286, 90.515     ,
       90.44285714, 90.37071429, 90.29857143, 90.22642857, 90.15428571,
       90.08214286, 90.01      , 89.93785714, 89.86571429, 89.79357143,
       89.72142857, 89.64928571, 89.57714286, 89.505     , 89.43285714,
       89.36071429, 89.28857143, 89.21642857, 89.14428571, 89.07

In [113]:
fig = px.line(
    df,
    x="date",
    y="weight",
    color="time_of_day",
    line_shape="spline",
    markers=True,
    text="weight",
)
fig.update_traces(textposition="bottom right")
fig.add_vline(
    x=diet_start,
    line_width=1,
    line_dash="dash",
    line_color="green",
    # annotation="Diet start",
)
fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="Expected loss rate"))
fig.add_trace(go.Scatter(x=df['date'], y=df['avg_daily'], mode='lines+markers', connectgaps=True, line_shape='spline', name="Average daily weight"))
fig.update_layout(
    xaxis_range=[
        df["date"][0] - datetime.timedelta(days=1),
        df["date"][len(df) - 1] + datetime.timedelta(days=1),
    ],
    yaxis_range=[min(df['weight'] -1),max(df['weight'])+1]
)
fig.show()

In [114]:
df.iloc[-1]

date           2022-12-14
time_of_day       morning
weight               93.5
avg_daily             NaN
Name: 10, dtype: object

In [91]:
prev_day = None
avg_daily = []
for row in df.itertuples():
    curr_day = row.date
    curr_weight = row.weight
    if curr_day == prev_day:
        avg_daily.append((prev_weight + curr_weight)/2)
    else:
        avg_daily.append(None)
    prev_day = curr_day
    prev_weight = curr_weight

    print(row)

Pandas(Index=0, date=datetime.date(2022, 12, 8), time_of_day='morning', weight=93.6)
Pandas(Index=1, date=datetime.date(2022, 12, 8), time_of_day='evening', weight=94.3)
Pandas(Index=2, date=datetime.date(2022, 12, 9), time_of_day='morning', weight=92.4)
Pandas(Index=3, date=datetime.date(2022, 12, 10), time_of_day='evening', weight=94.4)
Pandas(Index=4, date=datetime.date(2022, 12, 11), time_of_day='morning', weight=93.6)
Pandas(Index=5, date=datetime.date(2022, 12, 11), time_of_day='evening', weight=95.6)
Pandas(Index=6, date=datetime.date(2022, 12, 12), time_of_day='morning', weight=93.9)
Pandas(Index=7, date=datetime.date(2022, 12, 12), time_of_day='evening', weight=94.8)
Pandas(Index=8, date=datetime.date(2022, 12, 13), time_of_day='morning', weight=93.5)
Pandas(Index=9, date=datetime.date(2022, 12, 13), time_of_day='evening', weight=94.6)
Pandas(Index=10, date=datetime.date(2022, 12, 14), time_of_day='morning', weight=93.5)


In [116]:
pd.DataFrame([(1,2), (3,4)], columns=['a', 'b'])

,a,b
0,1,2
1,3,4


In [93]:
df['avg_daily'] = avg_daily

In [94]:
df

,date,time_of_day,weight,avg_daily
0,2022-12-08,morning,93.6,NaN
1,2022-12-08,evening,94.3,93.95
2,2022-12-09,morning,92.4,NaN
3,2022-12-10,evening,94.4,NaN
4,2022-12-11,morning,93.6,NaN
5,2022-12-11,evening,95.6,94.60
6,2022-12-12,morning,93.9,NaN
7,2022-12-12,evening,94.8,94.35
8,2022-12-13,morning,93.5,NaN
9,2022-12-13,evening,94.6,94.05


In [111]:
f = go.Figure(data=go.Scatter(x=df['date'], y=df['avg_daily'], mode='lines+markers', connectgaps=True))
f.show()
